TODO: downweight zero sales and use frequency on test store-item pair to train

In [316]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from pandas.tseries.offsets import MonthEnd
import cPickle

In [2]:
def isConsecutive(seq):
    # if non-consecutive, need to re-map to consecutive number starting from 1
    uniq = pd.unique(seq)
    return len(uniq) == (uniq.max()-uniq.min() + 1)

In [3]:
def isUniqBigger(seq1,seq2):
    return set(seq1) >= set(seq2)

In [4]:
def reMapDF(df,cols):
    # remap cols in dataframe to consecutive integers starting from one
    for col in cols:
        uniq = pd.unique(df[col])
        dict_ = {item:i+1 for i,item in enumerate(uniq)}
        df = df.replace({col:dict_})
    return df

In [5]:
def dimentionDF(df,cols):
    return {col:len(set(df[col])) for col in cols}

In [6]:
def mergeFillCast(df1,df2,key):
    cols = df2.columns.values
    types = df2.dtypes.values
    dict_ = {col:type_ for col,type_ in zip(cols,types)}
    dfOut = pd.merge(df1, df2, how='left', on=key, 
             suffixes=('', '_y'), copy=True, indicator=False).fillna(0)
    dfOut[cols] = \
        dfOut[cols].astype(dict_)
    return dfOut

In [7]:
def mergeFillCastsss(df0,dfs,keys):
    for df,key in zip(dfs,keys):
        df0 = mergeFillCast(df0,df,key)
    return df0

In [8]:
types = {'id': 'int32',
         'item_nbr': 'int32',
         'store_nbr': 'int8',
         'unit_sales': 'float32',
         'onpromotion': bool}

In [9]:
train = pd.read_csv('train.csv',usecols=['date','item_nbr','store_nbr','unit_sales','onpromotion'],\
                    parse_dates=['date'],dtype=types, infer_datetime_format=True)

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
train = train.fillna(2,axis=1)
train.onpromotion = train.onpromotion.astype(np.int8)

In [11]:
train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-01,25,103665,7.0,2
1,2013-01-01,25,105574,1.0,2
2,2013-01-01,25,105575,2.0,2
3,2013-01-01,25,108079,1.0,2
4,2013-01-01,25,108701,1.0,2


In [50]:
test = pd.read_csv('test.csv',parse_dates=['date'],dtype=types, infer_datetime_format=True)
test = test.fillna(2,axis=1)
test.onpromotion = test.onpromotion.astype(np.int8)

Needs to map items2 before mapping item_nbr

In [12]:
items = pd.read_csv('items.csv')
stores = pd.read_csv('stores.csv')

In [13]:
items2 = reMapDF(items,['family','class'])
items2[['family','class','perishable']] = \
        items2[['family','class','perishable']].astype('int16')

In [14]:
stores2 = reMapDF(stores,['city', 'state', 'type'])
stores2 = stores2.astype('int8')

In [15]:
val = train[train.date >= '2017-07-31']

In [16]:
train = train[train.date < '2017-07-31']

In [17]:
item_uniq = pd.unique(train.item_nbr)
item_dict = {item:i+1 for i,item in enumerate(item_uniq)}
iter_mapping = lambda x: item_dict[x] if x in item_dict else 0

Create date related variables. Multiple holiday can be on the same date. Sample one holiday from many to create multiple holiday lookup.

In [299]:
holidays_events = pd.read_csv('holidays_events.csv',parse_dates=['date'],infer_datetime_format=True)

In [300]:
holidays_events_list = [holidays_events.groupby('date').apply(lambda x: x.sample(frac=1)).reset_index(drop=True)\
                                .groupby('date').first().reset_index() for _ in range(10)]

In [301]:
holidays_events_list2 = [reMapDF(holiday.drop('description',1),['type', 'locale', 'locale_name'])\
                        for holiday in holidays_events_list]

In [302]:
for holiday in holidays_events_list2:
    holiday[['type', 'locale', 'locale_name','transferred']] = \
        holiday[['type', 'locale', 'locale_name','transferred']].astype('int8')

In [303]:
dateVar = pd.DataFrame(pd.date_range('2013-01-01', '2017-08-31'),columns=['date'])

In [304]:
dateVar['dayOfWeek'] = dateVar.date.dt.dayofweek

In [305]:
dateVar['payDay'] = ((dateVar.date.dt.day == dateVar.date.dt.days_in_month) | \
                     (dateVar.date.dt.day == 15)) * 1

In [306]:
dateVar['month'] = dateVar.date.dt.month

In [307]:
dateVar['earthquake'] = (dateVar.date > '2016-04-16') & (dateVar.date <= '2016-04-24')

In [308]:
dateVar.columns.values

array(['date', 'dayOfWeek', 'payDay', 'month', 'earthquake'], dtype=object)

In [309]:
dateVar[['dayOfWeek', 'payDay', 'month', 'earthquake']] = \
    dateVar[['dayOfWeek', 'payDay', 'month', 'earthquake']].astype('int8')

In [310]:
oil = pd.read_csv('oil.csv',parse_dates=['date'],infer_datetime_format=True)

In [311]:
dateVar = pd.merge(dateVar,oil,'left','date').fillna(method='bfill')

In [312]:
dateVar_list = [pd.merge(dateVar,holiday,'left','date').fillna(0)\
               for holiday in holidays_events_list2]

In [315]:
dateVar_list[2].head()

,date,dayOfWeek,payDay,month,earthquake,dcoilwtico,type,locale,locale_name,transferred
0,2013-01-01,1,0,1,0,93.14,1.0,3.0,12.0,0.0
1,2013-01-02,2,0,1,0,93.14,0.0,0.0,0.0,0.0
2,2013-01-03,3,0,1,0,92.97,0.0,0.0,0.0,0.0
3,2013-01-04,4,0,1,0,93.12,0.0,0.0,0.0,0.0
4,2013-01-05,5,0,1,0,93.20,5.0,3.0,12.0,0.0


In [317]:
with open(r"dateVar.pickle", "wb") as output_file:
    cPickle.dump(dateVar_list, output_file)

In [318]:
with open(r"dateVar.pickle", "rb") as input_file:
    dateVar_list = cPickle.load(input_file)

Use the first date that store has sales on any items as the first date to densify the time series data.

In [8]:
def CreateData(data):
    SI_timeMinMax = data.groupby(['store_nbr','item_nbr'])['date'].agg([np.count_nonzero]).reset_index()
    SI_train_sales = data.groupby(['store_nbr','item_nbr'])[['date','unit_sales']].\
                    agg(lambda x: tuple(x)).reset_index()
    storeTime = data.groupby(['store_nbr'])['date'].agg([np.min,np.max]).reset_index()
    dfs = [items2,stores2,SI_train_sales,storeTime]
    keys = ['item_nbr','store_nbr',['item_nbr','store_nbr'],'store_nbr']
    SI_train = mergeFillCastsss(SI_timeMinMax,dfs,keys)
    SI_train['item_nbr'] = SI_train.item_nbr.map(iter_mapping)
    SI_train['sampleWeight'] = SI_train.count_nonzero/((SI_train.amax - SI_train.amin).dt.days)
    SI_train.drop('count_nonzero',1,inplace=True)
    return SI_train

In [47]:
trainRNN = CreateData(train)
valRNN = CreateData(val)

In [78]:
trainRNN['amin']=pd.to_datetime(trainRNN['amin'])
trainRNN['amax']=pd.to_datetime(trainRNN['amax'])

In [81]:
valRNN['amin']=pd.to_datetime(valRNN['amin'])
valRNN['amax']=pd.to_datetime(valRNN['amax'])

In [82]:
trainRNN.to_csv('trainRNN.csv',index=False)

In [83]:
valRNN.to_csv('valRNN.csv',index=False)

In [8]:
trainRNN = pd.read_csv('trainRNN.csv',engine='python')

In [9]:
valRNN = pd.read_csv('valRNN.csv',engine='python')

In [321]:
def RNN_generator(data,batchSize,seqSize,df,key,discreteList,shuffle=True,downSample=1):
    # return bool (if init should be reset) 
    # and a list [y (B,T),weight (B,T),Xcontinue of shape (B,T,2)] + [Xdiscrete] of shape (B,T) + [X] of shape (B,)
    # finetune by use store-item pair in testset only
    if shuffle:
        data = data.sample(frac=1).reset_index(drop=True)
    n = data.shape[0]

    for from_ in range(0,n-batchSize,batchSize):
        X = list(data.loc[from_:from_+batchSize-1,'store_nbr':'cluster'].values.astype(np.int32).T)
        weight = np.ones((batchSize,1),dtype=np.float32)
        weight[data.loc[from_:from_+batchSize-1,'perishable']==1] = 1.25
        for j,(y,w,Xdiscrete,Xcontinue) in enumerate(timeGenerator(\
                           data.loc[from_:from_+batchSize-1,'date':'amax'],seqSize,downSample,df,key,discreteList)):
            yield j==0, [y,weight*w,Xcontinue] + Xdiscrete + X
            
def timeGenerator(data,seqSize,downSample,df,key,discreteList):
    # df is time related variables like holiday and seasonality
    # returns y, weight, X_discrete, X_continue
    n = data.shape[0]
    data['curr'] = data.amin 
    sparse = pd.concat([data.apply(lambda x: pd.Series(x.date,name='date'),axis=1)\
                                    .stack().reset_index().drop(['level_1'],1),\
                        data.apply(lambda x: pd.Series(x.unit_sales),axis=1)\
                                    .stack().reset_index().drop(['level_0','level_1'],1)],1)
    sparse.columns = ['level','date','sales']
    while np.all(data.curr + pd.DateOffset(seqSize) <= data['amax']):
        dense = data.apply(lambda x:pd.Series(pd.date_range(x.curr,periods=seqSize+1)),axis=1)\
                            .stack().reset_index().drop(['level_1'],1)
        dense.columns = ['level','date']
        dense = pd.merge(pd.merge(dense, df[np.random.randint(10)], how='left', on=key),
                        sparse,how='left',on=['level','date']).fillna(0)
        dense_continue = dense[['dcoilwtico','sales']].values.astype(np.float32)\
                                  .reshape((n,seqSize+1,2))[:,:seqSize,:]
        dense_discrete = list(np.moveaxis(dense[discreteList].values.astype(np.int32)\
                                                .reshape((n,seqSize+1,len(discreteList)))[:,:seqSize,:]\
                                          ,2,0))
        y = dense['sales'].values.astype(np.float32).reshape((n,seqSize+1))[:,1:]
        weight = np.ones_like(y,dtype=np.float32)
        weight[y==0] = downSample
        yield y, weight, dense_discrete, dense_continue
        data.curr = data.curr + pd.DateOffset(seqSize)   

In [54]:
# of shape (B,T)
discreteList = ['dayOfWeek','payDay','month','earthquake','type','locale','locale_name','transferred']

In [118]:
# of shape (B,)
Xlist = ['store_nbr','item_nbr',
 'family', 'class',
 'perishable', 'city',
 'state', 'type', 'cluster']

In [366]:
g = RNN_generator(trainRNN,5,3,dateVar_list,'date',discreteList,shuffle=True,downSample=0.8)

In [376]:
a,b = g.next()

In [377]:
a

False

In [378]:
b[0]

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]], dtype=float32)

In [379]:
b[1]

array([[ 0.80000001,  0.80000001,  0.80000001],
       [ 0.80000001,  0.80000001,  0.80000001],
       [ 1.        ,  1.        ,  1.        ],
       [ 0.80000001,  0.80000001,  0.80000001],
       [ 0.80000001,  0.80000001,  0.80000001]], dtype=float32)

In [380]:
b[2][:,:,0]

array([[ 93.12000275,  93.19999695,  93.19999695],
       [ 38.22000122,  39.15000153,  38.5       ],
       [ 93.19999695,  93.19999695,  93.19999695],
       [ 93.19999695,  93.19999695,  93.19999695],
       [ 93.19999695,  93.19999695,  93.19999695]], dtype=float32)

In [381]:
b[2][:,:,1]

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]], dtype=float32)

In [382]:
b[3]

array([[4, 5, 6],
       [0, 1, 2],
       [5, 6, 0],
       [5, 6, 0],
       [5, 6, 0]], dtype=int32)

In [383]:
b[10]

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int32)

In [384]:
b[11:]

[array([25, 42, 38, 54, 38], dtype=int32),
 array([3834, 1694, 1732, 1170,  947], dtype=int32),
 array([9, 1, 6, 1, 1], dtype=int32),
 array([333,  26,  18,  22,   7], dtype=int32),
 array([0, 0, 1, 0, 0], dtype=int32),
 array([11, 17, 18, 22, 18], dtype=int32),
 array([10, 12, 13, 16, 13], dtype=int32),
 array([1, 1, 1, 3, 1], dtype=int32),
 array([1, 2, 4, 3, 4], dtype=int32)]